<a href="https://colab.research.google.com/github/ngwalker93/ADS-504-Final-Team-Project/blob/main/ADS_504FinalProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 504 Final Team Project
# Weight Lifting Exercises monitored with Inertial Measurement Units

### Team 6: Kiara Paz, Jordan Torres, and Nancy Walker

## Introduction
 This project aims to assess the application of machine learning methods and programming techniques.

## Dataset Discription
Weight Lifting Exercises monitored with Inertial Measurement Units: https://archive.ics.uci.edu/dataset/273/weight+lifting+exercises+monitored+with+inertial+measurement+units

The data set was sourced from the UC Irvine Machine Learning Repository. It contains observations of six healthy subjects that performed 5 variations of the bicep curl weight lifting exercise. A health care professional predicted one of the varaitions.

## Problem Definition


## Import Libraries and Dataset

In [ ]:
# Import Libaries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Attach GitHub repository to retrieve dataset
!git clone https://github.com/ngwalker93/ADS-504-Final-Team-Project.git

Cloning into 'ADS-504-Final-Team-Project'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 473.92 KiB | 7.64 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
# Check that repository was added
!ls ADS-504-Final-Team-Project

ADS_504FinalProj.ipynb
Example_WearableComputing_weight_lifting_exercises_biceps_curl_variations.csv
README.md


In [ ]:
# Read .csv file
df = pd.read_csv('ADS-504-Final-Team-Project/Example_WearableComputing_weight_lifting_exercises_biceps_curl_variations.csv', skiprows=1, delimiter=',')
df.head()

,user_name,raw_timestamp_part_1,raw_timestamp_part_2,cvtd_timestamp,new_window,num_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,...,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,classe
0,eurico,1322489729,34670,28/11/2011 14:15,no,1,3.70,41.6,-82.8,3,...,-0.05,-0.37,-0.43,-170,155,184,-1160,1400,-876,E
1,eurico,1322489729,62641,28/11/2011 14:15,no,1,3.66,42.8,-82.5,2,...,-0.06,-0.37,-0.59,-178,164,182,-1150,1410,-871,E
2,eurico,1322489729,70653,28/11/2011 14:15,no,1,3.58,43.7,-82.3,1,...,-0.05,-0.27,-0.72,-182,172,185,-1130,1400,-863,E
3,eurico,1322489729,82654,28/11/2011 14:15,no,1,3.56,44.4,-82.1,1,...,0.02,-0.24,-0.79,-185,182,188,-1120,1400,-855,E
4,eurico,1322489729,90637,28/11/2011 14:15,no,1,3.57,45.1,-81.9,1,...,0.08,-0.27,-0.82,-188,195,188,-1100,1400,-843,E


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Columns: 159 entries, user_name to classe
dtypes: float64(103), int64(31), object(25)
memory usage: 4.9+ MB


## Exploratory Data Analysis (EDA)

In [ ]:
# Get the count of null values for each column
null_counts = df.isnull().sum()

# Display the null counts
display(null_counts[null_counts > 0]) # Display only columns with null values

,0
kurtosis_roll_belt,3936
kurtosis_picth_belt,3936
kurtosis_yaw_belt,3936
skewness_roll_belt,3936
skewness_roll_belt.1,3936
...,...
stddev_pitch_forearm,3936
var_pitch_forearm,3936
avg_yaw_forearm,3936
stddev_yaw_forearm,3936


Given that 100 columns have a large number of null values (3936 out of 4024 rows), dropping columns might be a reasonable approach if those specific features are not crucial for analysis. Alternatively, imputation can be consider, but with such a high percentage of missing values, imputation might not be the best option unless there is a strong reason to keep these columns.

Theses column appear to be the columns that contain statisitcal caculations. "Each IMU has x, y, and z values + euler angles (roll, pitch and yaw). For each time window (1s of data), there are several statistics calculations, like Kurtosis, Variance, etc."

In [ ]:
# Get the names of columns with null values
cols_to_drop = null_counts[null_counts > 0].index

# Drop the columns with null values
df_dropped = df.drop(columns=cols_to_drop)

# Display the first few rows and the columns and their data types of the new DataFrame
df_dropped.head()
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 59 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_name             4024 non-null   object 
 1   raw_timestamp_part_1  4024 non-null   int64  
 2   raw_timestamp_part_2  4024 non-null   int64  
 3   cvtd_timestamp        4024 non-null   object 
 4   new_window            4024 non-null   object 
 5   num_window            4024 non-null   int64  
 6   roll_belt             4024 non-null   float64
 7   pitch_belt            4024 non-null   float64
 8   yaw_belt              4024 non-null   float64
 9   total_accel_belt      4024 non-null   int64  
 10  gyros_belt_x          4024 non-null   float64
 11  gyros_belt_y          4024 non-null   float64
 12  gyros_belt_z          4024 non-null   float64
 13  accel_belt_x          4024 non-null   int64  
 14  accel_belt_y          4024 non-null   int64  
 15  accel_belt_z         

## Pre-Processing

## Modeling Methods, Validation, and Performance Metrics

### Decision Tree

### Linear Modeling

### Ensemble Modeling

### Kernal Modeling

### Neural Networks

### Support Vector Machines

## Modeling Results and Findings